In [1]:
import pandas as pd
import sqlalchemy as sa
import urllib

In [2]:
params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()


In [3]:
df_gene_ids = pd.read_sql(sa.text('SELECT $node_id, ensembl_id FROM gene_nodes'), cnxn)
df_gene_ids

,$node_id_70F4A652A18D4EAA817A05B8307F4E47,ensembl_id
0,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000419539
1,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000474279
2,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000347168
3,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000347168
4,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000347168
...,...,...
44827,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485663
44828,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485663
44829,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485663
44830,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485668


In [4]:
eid2nid_map = {}
for idx, row in df_gene_ids.iterrows():
    nid = row.iloc[0]
    eid = row['ensembl_id']
    eid2nid_map[eid] = nid

eid2nid_map

{'ENSP00000419539': '{"type":"node","schema":"dbo","table":"gene_nodes","id":0}',
 'ENSP00000474279': '{"type":"node","schema":"dbo","table":"gene_nodes","id":1}',
 'ENSP00000347168': '{"type":"node","schema":"dbo","table":"gene_nodes","id":58}',
 'ENSP00000305847': '{"type":"node","schema":"dbo","table":"gene_nodes","id":63}',
 'ENSP00000431814': '{"type":"node","schema":"dbo","table":"gene_nodes","id":64}',
 'ENSP00000328270': '{"type":"node","schema":"dbo","table":"gene_nodes","id":69}',
 'ENSP00000359096': '{"type":"node","schema":"dbo","table":"gene_nodes","id":71}',
 'ENSP00000424708': '{"type":"node","schema":"dbo","table":"gene_nodes","id":72}',
 'ENSP00000367164': '{"type":"node","schema":"dbo","table":"gene_nodes","id":74}',
 'ENSP00000375232': '{"type":"node","schema":"dbo","table":"gene_nodes","id":79}',
 'ENSP00000354694': '{"type":"node","schema":"dbo","table":"gene_nodes","id":80}',
 'ENSP00000458772': '{"type":"node","schema":"dbo","table":"gene_nodes","id":81}',
 'ENSP

In [5]:
df_edges = pd.read_csv("../parser/temp/gene_edges.tsv", sep="\t")
df_edges

,src,dst,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,ENSP00000000233,ENSP00000379496,0,0,0,54,0,0,144,155
1,ENSP00000000233,ENSP00000314067,0,0,0,0,180,0,61,197
2,ENSP00000000233,ENSP00000263116,0,0,0,62,152,0,101,222
3,ENSP00000000233,ENSP00000361263,0,0,0,0,161,0,64,181
4,ENSP00000000233,ENSP00000409666,0,0,0,82,213,0,72,270
...,...,...,...,...,...,...,...,...,...,...
11861071,ENSP00000485678,ENSP00000354800,0,0,0,213,0,0,0,213
11861072,ENSP00000485678,ENSP00000308270,0,0,0,152,0,0,0,151
11861073,ENSP00000485678,ENSP00000335660,0,0,0,182,0,0,0,181
11861074,ENSP00000485678,ENSP00000300127,0,0,0,155,0,0,0,154


In [6]:
df = df_edges.copy()
df['$from_id'] = [eid2nid_map[i] for i in df['src'].to_list()]
df['$to_id'] = [eid2nid_map[i] for i in df['dst'].to_list()]
df

,src,dst,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,$from_id,$to_id
0,ENSP00000000233,ENSP00000379496,0,0,0,54,0,0,144,155,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
1,ENSP00000000233,ENSP00000314067,0,0,0,0,180,0,61,197,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
2,ENSP00000000233,ENSP00000263116,0,0,0,62,152,0,101,222,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
3,ENSP00000000233,ENSP00000361263,0,0,0,0,161,0,64,181,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
4,ENSP00000000233,ENSP00000409666,0,0,0,82,213,0,72,270,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
...,...,...,...,...,...,...,...,...,...,...,...,...
11861071,ENSP00000485678,ENSP00000354800,0,0,0,213,0,0,0,213,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
11861072,ENSP00000485678,ENSP00000308270,0,0,0,152,0,0,0,151,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
11861073,ENSP00000485678,ENSP00000335660,0,0,0,182,0,0,0,181,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."
11861074,ENSP00000485678,ENSP00000300127,0,0,0,155,0,0,0,154,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no..."


In [8]:
df.columns

Index(['src', 'dst', 'neighborhood', 'fusion', 'cooccurence', 'coexpression',
       'experimental', 'database', 'textmining', 'combined_score', '$from_id',
       '$to_id'],
      dtype='object')

In [9]:
df = df[['$from_id', '$to_id','src', 'dst', 'neighborhood', 'fusion', 'cooccurence', 'coexpression',
       'experimental', 'database', 'textmining', 'combined_score']]
df

,$from_id,$to_id,src,dst,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000000233,ENSP00000379496,0,0,0,54,0,0,144,155
1,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000000233,ENSP00000314067,0,0,0,0,180,0,61,197
2,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000000233,ENSP00000263116,0,0,0,62,152,0,101,222
3,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000000233,ENSP00000361263,0,0,0,0,161,0,64,181
4,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000000233,ENSP00000409666,0,0,0,82,213,0,72,270
...,...,...,...,...,...,...,...,...,...,...,...,...
11861071,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485678,ENSP00000354800,0,0,0,213,0,0,0,213
11861072,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485678,ENSP00000308270,0,0,0,152,0,0,0,151
11861073,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485678,ENSP00000335660,0,0,0,182,0,0,0,181
11861074,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",ENSP00000485678,ENSP00000300127,0,0,0,155,0,0,0,154


In [10]:
df.to_csv("./ingest/gene_gene_edges_ids.csv", sep='\t', index=False)